In [ ]:
import time
import matplotlib.pyplot as plt
from generate_data import *
from queue import PriorityQueue
import pandas as pd
import numpy as np
from matplotlib.animation import FuncAnimation
from matplotlib.animation import FuncAnimation, PillowWriter 

start = str(input('start warehouse:'))
end = str(input('end warehouse:'))

list_mw_north, list_mw_central, list_mw_south = create_domain_partition()
[north_mw,central_mw,south_mw],data = create_data_UCS()

#phần này tạo data cho visualize
df = pd.read_excel('Data_Excel.xlsx')
lat = {}
long = {}
for i in range(len(df)):
    lat[df.iloc[i][0].strip()] = df.iloc[i][64]
    long[df.iloc[i][0].strip()] = df.iloc[i][65]
    
longtitude = []
latitude = []
province = []
for i in range(len(df)):
    longtitude.append(df.iloc[i][65])
    latitude.append(df.iloc[i][64])
    province.append(df.iloc[i][0])
    
start_time = time.time()
class node():
    def __init__ (self, name = 0, par = None, g = 0):
        self.name = name
        self.par = par
        self.g = g
    def __lt__(self, other):
        return self.g < other.g
    def __eq__(self,other):
        if other == None:
            return False
        else:
            return self.name == other.name
    def parent(self):
        print(self.name,'cost',self.g)
        if self.par != None:
            return self.par.parent()

def DFS (S = node(start), G = node(end)):
    print('DFS(',start,end,')')
    open = []
    closed = []
    S.g = 0
    open.append(S)
    i = 0
    while True:
        i+=1
        #print('STEP ',i,': open = ',[x.name for x in open])
        if len(open) == 0:
            print("can't solve")
            break
        else:
            O = open.pop(0)
            closed.append(O)
            lat_check.append(lat[O.name])  #add to expanded node to visualize after
            long_check.append(long[O.name]) #add to expanded node to visualize after
            if len(data[O.name])!=0:
                pass
                #print('Scan',O.name)
                #print()
            if O.name == G.name:
                print("\033[1m" + 'solve successfully' + "\033[0m")
                print()
                O.parent()        
                break
            else:
                pos = 0
                for i in range(0,len(data[O.name]),2):
                    tmp = node(data[O.name][i])
                    tmp.g = O.g + data[O.name][i+1]
                    tmp.par = O
                    if tmp not in open and tmp not in closed:
                        open.insert(pos,tmp)
                        pos+=1
                                
if start in list_mw_south:
    required_warehouse = south_mw
elif start in list_mw_north:
    required_warehouse = north_mw
elif start in list_mw_central:
    required_warehouse = central_mw
        
def check_same_area_start_warehouse(start,end):   #check if start warehouse and end warehouse are in the same area or not
    if start in list_mw_central and end in list_mw_central:
        return True
    elif start in list_mw_north and end in list_mw_north:
        return True
    elif start in list_mw_south and end in list_mw_south:
        return True
    return False

def distance(provinceA, provinceB):   #return distance between 2 warehouse
    for i in range(len(df)):
        if df.iloc[i,0] == provinceA:
            return df[provinceB].iloc[i]
def find_max_distance(provinceA, provinceB,provinceC):  #find longest edge of triangle
    a = distance(provinceA, provinceB)
    b = distance(provinceB, provinceC)
    c = distance(provinceC, provinceA)
    if a*a >= b*b + c*c:
        return a
    elif b*b >= c*c + a*a:
        return b
    elif c*c >= a*a + b*b:
        return c
    
def rectangular_check(): #with different algorithm, we just need to change algorithm function. E.g: AStar() --> UCS()
    if not check_same_area_start_warehouse(start,end):    #if 2 warehouse are in different area then start warehouse must transfer goods to its area mainwarehouse before shipping to end ware house
        DFS(S = node(start), G = node(required_warehouse))
        DFS(S = node(required_warehouse), G = node(end))
    else:
        if distance(start,end) == find_max_distance(start,end,required_warehouse) and start != required_warehouse and end != required_warehouse: 
            DFS(S = node(start), G = node(required_warehouse))
            DFS(S = node(required_warehouse), G = node(end))
        else:
            DFS(S = node(start), G = node(end))

def save_path():  #this function save a gif picture represent scanned warehouse to find optimal path
    %matplotlib inline
    fig = plt.figure()
    fig,axes = plt.subplots(figsize = (10,19))
    for j in range (1,len(latitude)):
        axes.scatter(longtitude[j], latitude[j], c = 'lightblue')
        axes.text(longtitude[j], latitude[j],province[j])
    line, = axes.plot([],[],color = '#33FF00',marker='.',markersize=30)
    def animate(i):
        x = long_check[i]
        y = lat_check[i]
        print(x,y)
        line.set_data(x,y)
        return line,
    anim = FuncAnimation(fig, animate, frames = len(long_check), interval = 300, blit = True)
    anim.save('path.gif', writer = 'pillow')
    
lat_check = []
long_check = []

rectangular_check()

print('time',time.time()-start_time,'s')